In [166]:
from feedparser.util import FeedParserDict
import feedparser
import pandas as pd
from tqdm import tqdm
import bibtexparser
import numpy as np
import pprint
import copy
import json

In [ ]:
#LOAD DATASET

dois:FeedParserDict = feedparser.parse(open("Arxiv Search.txt", 'r', encoding='utf-8').read())

keys = dois.keys()
print(keys)

dict_keys(['bozo', 'entries', 'feed', 'headers', 'encoding', 'version', 'bozo_exception', 'namespaces'])


In [17]:
#EXAMPLE
pprint.pprint(dois['entries'][0])

{'arxiv_comment': '14 pages, 6 figures. Published at Springer Neural Computing '
                  'and\n'
                  '  Applications journal',
 'arxiv_doi': '10.1007/s00521-022-08068-6',
 'arxiv_journal_ref': 'Neural Comput & Applic (2022)',
 'arxiv_primary_category': {'scheme': 'http://arxiv.org/schemas/atom',
                            'term': 'cs.CL'},
 'author': 'João Baptista de Oliveira e Souza Filho',
 'author_detail': {'name': 'João Baptista de Oliveira e Souza Filho'},
 'authors': [{'name': 'Frederico Dias Souza'},
             {'name': 'João Baptista de Oliveira e Souza Filho'}],
 'guidislink': True,
 'id': 'http://arxiv.org/abs/2212.00587v1',
 'link': 'http://arxiv.org/abs/2212.00587v1',
 'links': [{'href': 'http://dx.doi.org/10.1007/s00521-022-08068-6',
            'rel': 'related',
            'title': 'doi',
            'type': 'text/html'},
           {'href': 'http://arxiv.org/abs/2212.00587v1',
            'rel': 'alternate',
            'type': 'text/html'},


In [ ]:
#SAVE INDEX AND DOI ONLY
results2doi = pd.DataFrame(columns=['idx', 'doi'])

i = 0    
with tqdm(total=len(dois['entries'])) as pbar:
    for entry in dois['entries']:
        try:                    
            link = entry['id']            
            doi:str = link[link.rfind('/')+1:]            
            results2doi = pd.concat([results2doi, pd.DataFrame([[i, doi]], columns=results2doi.columns)])
            pbar.update()
            i+=1
        except Exception as e:
            pass
results2doi.to_csv("./Arxiv_doi.csv", index=False, mode='w')

In [79]:
#REMOVE DUPLICATES
arxiv_doi = pd.read_csv('./Arxiv_doi.csv')

copia = arxiv_doi.copy(deep=True)
copia['doi'] = copia['doi'].apply(lambda x: x[:-2])
copia.drop_duplicates('doi', inplace=True)

arxiv_doi = arxiv_doi.merge(copia, how='inner', on="idx")
arxiv_doi.drop('doi_y', axis=1, inplace=True)

arxiv_doi['doi'] = arxiv_doi['doi_x']
arxiv_doi.drop('doi_x', axis=1, inplace=True)

arxiv_doi.to_csv('./no_duplicates_doi.csv', index=False, mode='w')

In [104]:
#GET ITEMS FROM DOI
no_dup = pd.read_csv('./no_duplicates_doi.csv')
copia = []
for i in range(len(dois['entries'])):
    if i in no_dup['idx']:
        copia.append(dois['entries'][i])    

In [110]:
#REMOVE PUBLISHED BEFORE 2015
rem = []
for entry in dois['entries']:
    if entry['published_parsed'].tm_year < 2015:
        rem+= [entry]
for r in rem:
    try: 
        copia.remove(r)
    except:
        continue
        

In [169]:
copia = [json.loads(json.dumps(x)) for x in copia]
doi_codes = []
for entry in copia:
    doi_codes+= [entry['link'][entry['link'].rfind('/') + 1:]]

pd.DataFrame(doi_codes).to_csv("doi2bib.csv", index=False, header=None)

In [161]:
import json

In [167]:
json.loads(json.dumps(copia[0]))

{'id': 'http://arxiv.org/abs/2212.00587v1',
 'guidislink': True,
 'link': 'http://arxiv.org/abs/2212.00587v1',
 'updated': '2022-12-01T15:24:19Z',
 'updated_parsed': [2022, 12, 1, 15, 24, 19, 3, 335, 0],
 'published': '2022-12-01T15:24:19Z',
 'published_parsed': [2022, 12, 1, 15, 24, 19, 3, 335, 0],
 'title': 'Embedding generation for text classification of Brazilian Portuguese\n  user reviews: from bag-of-words to transformers',
 'title_detail': {'type': 'text/plain',
  'language': None,
  'base': '',
  'value': 'Embedding generation for text classification of Brazilian Portuguese\n  user reviews: from bag-of-words to transformers'},
 'summary': 'Text classification is a natural language processing (NLP) task relevant to\nmany commercial applications, like e-commerce and customer service. Naturally,\nclassifying such excerpts accurately often represents a challenge, due to\nintrinsic language aspects, like irony and nuance. To accomplish this task, one\nmust provide a robust numerical

In [170]:
print(len(no_dup['idx']))
print(len(copia))
print(len(no_dup['idx']) - len(copia))
print(len(rem))

17249
14968
2281
12335


In [141]:
#bib_database = bibtexparser.load(bibtex_file=open("./k2.bib", 'r'))
#df = pd.DataFrame(bib_database.entries)